In [10]:
from qutip import *
import numpy as np
import math
import random
z0 = basis(2,0)
z1 = basis(2,1)
# We create the four Bell states.
st0 = (tensor(z0,z0)+tensor(z1,z1))/math.sqrt(2)
st1 = (tensor(z0,z0)-tensor(z1,z1))/math.sqrt(2)
st2 = (tensor(z0,z1)+tensor(z1,z0))/math.sqrt(2)
st3 = (tensor(z0,z1)-tensor(z1,z0))/math.sqrt(2)
print(st0)

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[ 0.70710678]
 [ 0.        ]
 [ 0.        ]
 [ 0.70710678]]


In [62]:
# This implements superdense coding.
# Alice's message can be 0,1,2,3.
a=random.randint(0,3)
# Choose the operation based on Alice's message.
if a==0:
    op=identity(2)
elif a==1:
    op=sigmaz()
elif a==2:
    op=sigmax()
else:
    op=sigmaz()*sigmax()
# The initial Bell state is now operated on by Alice's choice of operator.
state=tensor(op,identity(2))*st0
# Calculate the probabilities of the measurement results.
pr0=((st0.dag()*state).norm())**2
pr1=((st1.dag()*state).norm())**2
pr2=((st2.dag()*state).norm())**2
pr3=((st3.dag()*state).norm())**2
# Generate a random number.
rnd=random.random()
# Give a result according to the probability distribution.
if pr0>rnd:
    s=0
elif pr0+pr1>rnd:
    s=1
elif pr0+pr1+pr2>rnd:
    s=2
else:
    s=3
print("The input number is: ", a)
print("The number communicated is: ", s)

The input number is:  2
The number communicated is:  2


In [73]:
# Next we implement teleportation.
# Set a random state:
psi=random.random()*z0+random.random()*z1  # This is real but we could make it complex.
psi=psi.unit()  # normalise it
print("The starting state is: ",psi*psi.dag())
# Add an entangled state.
state=tensor(st0,psi)
# Create a bunch of projectors for all of the Bell measurement results.
# Note that these act on one qubit in the entangled pair and the qubit with the state psi to be teleported.
prj0=tensor(identity(2),st0*st0.dag())
prj1=tensor(identity(2),st1*st1.dag())
prj2=tensor(identity(2),st2*st2.dag())
prj3=tensor(identity(2),st3*st3.dag())
# Calculate probabilities.
pr0=((prj0*state).norm())**2
pr1=((prj1*state).norm())**2
pr2=((prj2*state).norm())**2
pr3=((prj3*state).norm())**2
# Generate a random number.
rnd=random.random()
# Give a result according to the probability distribution.
if pr0>rnd:
    a=0
    state=prj0*state
elif pr0+pr1>rnd:
    a=1
    state=prj1*state
elif pr0+pr1+pr2>rnd:
    a=2
    state=prj2*state
else:
    a=3
    state=prj3*state
# Give the normalised state on qubit 0 (the first one).
state=(state.unit()).ptrace(0)
print("The measurement result is: ",a)
print("The state is now: ",state)
# Set Alice's correction operation.
if a==0:
    op=identity(2)
elif a==1:
    op=sigmaz()
elif a==2:
    op=sigmax()
else:
    op=sigmaz()*sigmax()
# Correct the state.
state=op*state*op.dag()
print("The corrected state is: ",state)
print("The error is: ",state-psi*psi.dag())

The starting state is:  Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 0.56026352  0.49635502]
 [ 0.49635502  0.43973648]]
The measurement result is:  1
The state is now:  Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 0.56026352 -0.49635502]
 [-0.49635502  0.43973648]]
The corrected state is:  Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 0.56026352  0.49635502]
 [ 0.49635502  0.43973648]]
The error is:  Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 0.  0.]
 [ 0.  0.]]


In [85]:
# Now we combine the two. We do superdense coding then teleport one of the qubits.
# Alice's message can be 0,1,2,3.
a=random.randint(0,3)
# Choose the operation based on Alice's message.
if a==0:
    op=identity(2)
elif a==1:
    op=sigmaz()
elif a==2:
    op=sigmax()
else:
    op=sigmaz()*sigmax()
# The initial Bell state is now operated on by Alice's choice of operator.
state=tensor(op,identity(2))*st0
# Now we add an entangled pair.  The first two qubits are the new entangled pair for teleportation,
# and the second two qubits are the entangled pair for teleportation.
state=tensor(st0,state)
# We now want to do a Bell measurement on the middle two qubits, and define the projectors.
prj0=tensor(identity(2),st0*st0.dag(),identity(2))
prj1=tensor(identity(2),st1*st1.dag(),identity(2))
prj2=tensor(identity(2),st2*st2.dag(),identity(2))
prj3=tensor(identity(2),st3*st3.dag(),identity(2))
# The next part is just the same as before.
# Calculate probabilities.
pr0=((prj0*state).norm())**2
pr1=((prj1*state).norm())**2
pr2=((prj2*state).norm())**2
pr3=((prj3*state).norm())**2
# Generate a random number.
rnd=random.random()
# Give a result according to the probability distribution.
if pr0>rnd:
    s=0
    state=prj0*state
elif pr0+pr1>rnd:
    s=1
    state=prj1*state
elif pr0+pr1+pr2>rnd:
    s=2
    state=prj2*state
else:
    s=3
    state=prj3*state
# Give the normalised state on qubit 0 (the first one).
state=(state.unit()).ptrace([0,3])
# Set Alice's correction operation.
if s==0:
    op=identity(2)
elif s==1:
    op=sigmaz()
elif s==2:
    op=sigmax()
else:
    op=sigmaz()*sigmax()
op=tensor(op,identity(2))
# Correct the state.
state=op*state*op.dag()
# Calculate the probabilities of the measurement results.
pr0=((st0.dag()*state).norm())**2
pr1=((st1.dag()*state).norm())**2
pr2=((st2.dag()*state).norm())**2
pr3=((st3.dag()*state).norm())**2
# Generate a random number.
rnd=random.random()
# Give a result according to the probability distribution.
if pr0>rnd:
    s=0
elif pr0+pr1>rnd:
    s=1
elif pr0+pr1+pr2>rnd:
    s=2
else:
    s=3
print("The input number is: ", a)
print("The number communicated is: ", s)

The input number is:  1
The number communicated is:  1
